In [3]:
import sys
!{sys.executable} -m pip install bs4
!{sys.executable} -m pip install lxml

     |████████████████████████████████| 122kB 20.5MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
     |████████████████████████████████| 5.5MB 6.4MB/s eta 0:00:01


In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from lxml import html
from urllib.request import urlopen
import json

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
df_canada = pd.read_html(url, header=0)[0]

df_canada.head(10)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [10]:
df_canada = df_canada[df_canada['Borough'] != 'Not assigned']
df_canada

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [11]:
df_canada.shape # removing not assigned items from

(103, 3)

In [9]:
!{sys.executable} -m pip install geocoder
import geocoder

     |████████████████████████████████| 102kB 6.0MB/s ta 0:00:011


lat_lng_cords = None
lat_list = []
long_list = []

for index, row in df_canada.iterrows():
    postal_code = row["Postal Code"]
    while (lat_lng_cords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        print(g)
        lat_lng_coords = g.latlng
        print(lat_lng_cords)
        
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    lat_list.append(latitude)
    long_list.append(longitude)
    lat_lng_cords = None
    
print(lat_list)
print(long_list)

In [9]:
df_coords = pd.read_csv('Geospatial_Coordinates.csv')

df_coords.shape

(103, 3)

In [12]:
df_canada = df_canada.merge(df_coords, on = "Postal Code", how = "inner")
df_canada

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
